In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import h5py
import pandas as pd
from astropy.table import Table


In [ ]:
file_z = [0.5, 0.7, 1.0, 1.5, 2.0] # All redshifts for DESI MgII simulated absorbers from SALSA
for this_z in file_z:
    print('Processing z = ' + str(this_z) + '...')
    file_in = 'spectra_TNG50-1_z{z}_n1000d2-fullbox_DESI_MgII_combined.hdf5'.format(z=this_z) # file name for the input hdf5 file containing the simulated spectra and absorber properties
    file_out = 'Tau_MgII_z{z}.hdf5'.format(z=this_z) # output file name for tau and wave data
    fits_out = 'MgII_Simulated_Catalog_z{z}.fits'.format(z=this_z) # output fits catalog
    with h5py.File(file_in,'r') as f_in:
        wave_array = f_in['wave'][:] # wavelegnth range same for all files, same as DESI range
        nabs = int(1e4); # total absorbers we want for each redshift
        all_EW_2796 = f_in['EW_MgII_2796'][:] # Equivelent width of the  2796 line
        valid_ind = [ind for ind, val in enumerate(all_EW_2796) if val > 0.05/(1 + this_z)] # Only consider absorbers with non-zero REW
        valid_abs = len(valid_ind)
        if valid_abs < nabs: # limits max absorbers to total number of valid absorbers in the file
            nabs = valid_abs
        valid_ind = valid_ind[:nabs] # Limit to n valid absorbers
        
        EW_2796_array = all_EW_2796[valid_ind] # Equivelent width of the  2796 line
        EW_2803_array = f_in['EW_MgII_2803'][valid_ind] # Equivelent width of the  2803 line
        N_2796_array = f_in['N_MgII_2796'][valid_ind] # Column density of the 2796 line
        N_2803_array = f_in['N_MgII_2803'][valid_ind] # Column density of the 2803 line
        tau_2796_array = f_in['tau_MgII_2796'][valid_ind] # optical depth array of the 2796 line
        tau_2803_array = f_in['tau_MgII_2803'][valid_ind] # optical depth array of the 2803 line
        tau_data = tau_2796_array + tau_2803_array # Total optical depth array for both lines
        z_array = wave_array[np.argmax(tau_2796_array[:], axis=1)] / 2796.35 - 1 # Calculate redshift of each absorber

        # Save tau and wave data to HDF5 file
        with h5py.File(file_out, "w") as f:
            f.create_dataset("wave", data=wave_array) 
            dset = f.create_dataset(
                "tau_MgII",
                shape=(nabs, len(wave_array)),
                dtype=np.float32,
                compression="gzip",
                compression_opts=4,
                shuffle=True,
                data=tau_data
                )
                
    # Create and Save Catalog to FITS file  
    t = Table()
    t['Sim_Index'] = valid_ind
    t['Z'] = z_array
    t['EW_2796'] = EW_2796_array
    t['EW_2803'] = EW_2803_array
    t['N_2796'] = N_2796_array
    t['N_2803'] = N_2803_array
    t.write(fits_out, format='fits', overwrite=True)